#  Intel® AI Reference Models Jupyter Notebook

This Jupyter notebook helps you choose and run a comparison between two models from the Intel® AI Reference Models repo using Intel® Optimizations for TensorFlow*, Intel® extension for TensorFlow, Intel® Optimizations for PyTorch and Intel® extension for PyTorch. When you run the notebook, it installs required package dependencies, displays information about your platform, lets you choose the two models to compare, runs those models, and finally displays a performance comparison chart.

<a id='section_1'></a>
# Step 1: Display Platform Information 

Install the required dependencies for the jupyter notebook

In [ ]:

!pip install matplotlib ipykernel psutil pandas cxxfilt gitpython
!pip install gcg
!python3 -m pip install gitpython
!pip install prettytable
!pip install --upgrade matplotlib
!pip install ipywidgets

Select a framework from the dropdown option below the next cell and install the required framework packages

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os

os.environ['no_proxy'] = ""
os.environ['NO_PROXY'] =""

# Create a dropdown widget without a default value
framework_dropdown = widgets.Dropdown(
    options=['PyTorch', 'TensorFlow'],
    value=None,
    description='Choose Framework:',
    disabled=False,
)

# Display the dropdown widget
display(framework_dropdown)

def run_command(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    return stdout.decode(), stderr.decode(), process.returncode

def install_packages(selected_framework):
    if selected_framework == 'PyTorch':
        # Install the latest version.
        # Installation 2a. pypi.org
        subprocess.run(["python", "-m", "pip", "install", "torch", "--upgrade", "--index-url", "https://download.pytorch.org/whl/cpu"])
        subprocess.run(["python", "-m", "pip", "install", "torchvision", "--upgrade", "--index-url", "https://download.pytorch.org/whl/cpu"])
        subprocess.run(["python", "-m", "pip", "install", "torchaudio", "--upgrade", "--index-url", "https://download.pytorch.org/whl/cpu"])
        subprocess.run(["python", "-m", "pip", "install", "intel-extension-for-pytorch", "--upgrade"])
        subprocess.run(["python", "-m", "pip", "install", "oneccl_bind_pt", "--upgrade", "--extra-index-url", "https://pytorch-extension.intel.com/release-whl/stable/cpu/us/"])

        print("PyTorch packages installed.")
        os.environ['Framework'] = 'pytorch'
        os.environ['yaml_file'] = 'profiling/ai_reference_models_pytorch.yaml'

        # Install dependencies for pytorch:
        print("Install PyTorch dependencies")
        current_directory = os.getcwd()

        if os.path.isdir("jemalloc"):
            import shutil
            shutil.rmtree("jemalloc")
            os.environ.pop("LD_PRELOAD", None)
            os.environ.pop("MALLOC_CONF", None)
        !git clone https://github.com/jemalloc/jemalloc.git
        os.chdir("jemalloc")
        !git checkout c8209150f9d219a137412b06431c9d52839c7272
        !./autogen.sh
        !./configure --prefix={current_directory}/
        !make
        !make install
        os.chdir("..")

        !pip install packaging intel-openmp --target f"{current_directory}"

        if os.path.isdir("gperftools-2.7.90"):
            shutil.rmtree("gperftools-2.7.90")
        !wget https://github.com/gperftools/gperftools/releases/download/gperftools-2.7.90/gperftools-2.7.90.tar.gz
        !tar -xzf gperftools-2.7.90.tar.gz
        os.chdir("gperftools-2.7.90")
        !./configure --prefix={current_directory}/tcmalloc
        !make
        !make install
        os.chdir("..")

        # print(f"{current_directory}/jemalloc/lib/libjemalloc.so:{current_directory}/tcmalloc/lib/libtcmalloc.so:/usr/local/lib/libiomp5.so")
        # os.environ["LD_PRELOAD"] = f"{current_directory}/jemalloc/lib/libjemalloc.so:{current_directory}/tcmalloc/lib/libtcmalloc.so:/usr/local/lib/libiomp5.so"
        os.environ["LD_PRELOAD"] = f"{current_directory}/jemalloc/lib/libjemalloc.so:{current_directory}/tcmalloc/lib/libtcmalloc.so"
        os.environ["MALLOC_CONF"] = "oversize_threshold:1,background_thread:true,metadata_thp:auto,dirty_decay_ms:9000000000,muzzy_decay_ms:9000000000"
        os.environ["DNNL_MAX_CPU_ISA"] = "AVX512_CORE_AMX"
        print("Finish building PyTorch")
    elif selected_framework == 'TensorFlow':
        subprocess.run(["pip", "install", "intel-tensorflow"])
        # subprocess.run(["pip", "install", "tensorflow"])
        subprocess.run(["pip", "install", "--upgrade", "intel-extension-for-tensorflow[cpu]"])
        print("TensorFlow packages installed.")
        os.environ['Framework'] = 'tensorflow'
        os.environ['yaml_file'] = 'profiling/ai_reference_models_tensorflow.yaml'
        print("Finish building TF")

    else:
        print("Invalid framework selected.")

# Add an event handler to the dropdown to trigger package installation
def on_dropdown_change(change):
    install_packages(change.new)

framework_dropdown.observe(on_dropdown_change, names='value')


Set the path for the AI reference models, assumed to be the current working directory, and the path for the utility functions directory inside the cloned Intel® AI Reference Models directory.

In [ ]:
# If default path does not work, change AIReferenceRoot path according to your environment
os.environ['NotebookRoot'] = os.getcwd()
os.environ['AIReferenceRoot'] = os.environ['NotebookRoot'] + os.sep + ".."
os.environ['ProfileUtilsRoot'] = os.environ['AIReferenceRoot'] + os.sep + "notebooks/profiling/"
print("Path for the AI Reference Models root is: ", os.environ['AIReferenceRoot'])
print("Path for utility functions directory is: ", os.environ['ProfileUtilsRoot'])

# Check for mandatory python scripts after AIReferenceRoot and ProfileUtilsRoot are assigned
import os
current_path = os.getcwd()
benchmark_path = os.environ['AIReferenceRoot'] + os.sep + "benchmarks/launch_benchmark.py"
if os.path.exists(benchmark_path) == True:
    print(benchmark_path)
else:
    print(benchmark_path)
    print("ERROR! Can't find benchmark/launch_benchmark.py script!")

profile_utils_path = os.environ['ProfileUtilsRoot'] + "profile_utils.py"
if os.path.exists(profile_utils_path) == True:
    print(profile_utils_path)
else:
    print("ERROR! Can't find profile_utils.py script!")

## Step 2: Run the first model

This notebook helps you compare the performance of two models listed in the supported models. Select the first model to run and compare.  (If the environment variable MODEL_1_INDEX is set, we'll use that instead of prompting for input.)

### Step 2.1: Display all the supported models and select first  model to run.

In [ ]:
notebook_root = os.environ['NotebookRoot']
%cd $notebook_root

After the list of models is displayed, select one by entering the model index number.

In [ ]:
from profiling.profile_utils import AIReferenceConfigFile
from prettytable import PrettyTable

config = AIReferenceConfigFile(os.environ['yaml_file'])
sections = config.read_supported_section()

# Create a table with headers
models_table = PrettyTable(["Index", "Model Name", "Framework", "Mode"])

# Iterate through the sections and add rows to the table
for index, section in enumerate(sections):
    model_name = section['model-name']
    framework = section['framework']
    mode = section['mode']

    # Add a row to the table
    models_table.add_row([index, model_name, framework, mode])

print("Supported Models: ")
print(models_table)

# use the "MODEL_1_INDEX" environment variable value if it exists.
import os
env_model_1_index=os.environ.get('MODEL_1_INDEX', '')
if env_model_1_index != '':
    model_1_index= int(env_model_1_index)
else:
    ## USER INPUT
    model_1_index= int(input('Input a index number of a model: '))

# List out the selected model name
if model_1_index >= len(sections):
    print("ERROR! Input a model_index value between 0 and ", len(sections))
else:
    model_1_name=sections[model_1_index]['name']

    # Prints out model name
    print("First model is: ", model_1_name.split()[0] )

    selected_section = sections[model_1_index]
    model_name_1 = selected_section['model-name']

    # Check if the selected model has a precision section
    if model_name_1 in selected_section:
        print(f"Available Precisions for {model_name_1}:")

        # Create a table with headers
        precision_table = PrettyTable(["Index", "Precision"])
        precisions = selected_section[model_name_1]

        # Iterate through the available precisions
        for index, precision_info in enumerate(selected_section[model_name_1]):
            precision = precision_info['precision']

            # Add a row to the table
            precision_table.add_row([index, precision])

        print(precision_table)

        # Prompt the user to select a precision
        precision_index = int(input('Select an index number for the precision: '))
        selected_precision = precisions[precision_index]['precision']
        print("Selected Precision for", model_1_name, selected_precision)

        # Print a table for available scripts for selected precision
        script_section = precisions[precision_index]
        script_name = script_section['script']

        print(f"Available Scripts for {selected_precision}:")

        # Create a table with headers
        script_table = PrettyTable(["Index", "Script"])

        # Iterate through the available scripts
        for index, script in enumerate(script_name):
            script_table.add_row([index, script])

        print(script_table)

        # Prompt the user to select a script
        script_index = int(input('Input a index number for the script: '))
        selected_row = script_table._rows[script_index]
        selected_script = selected_row[1]
        print("Selected script for", model_1_name, selected_script)

        # Prints out model name
        print("First model with selected precision and script is: ", model_1_name, selected_precision, selected_script)
        os.environ['PRECISION']=selected_precision



### Additional Setup
The following cell checks for additional environment variables that needs to be specified per model specification. 

Then checks for additional files, libraries, and configurations that needs to be installed per model specification. Commands are installed in a chronological dependency order.

In [ ]:
import subprocess
ai_reference_root = os.environ.get('AIReferenceRoot')
%cd $ai_reference_root

for export in selected_section['exports']:
    env_name = export.split(' ')[0]
    env_value = export.split(' ')[1]
    if env_value == '-i':
        env_value = input(f'Please provide the path for the following environment variable {env_name}: ', )
    os.environ[env_name] = os.path.expandvars(env_value)

    print("Assigned environment variable ", env_name, " : ", os.environ[env_name])

for cmd in selected_section['additional-commands']:
    subprocess.call(cmd, shell=True)

### Step 2.2: Get the required dataset for the selected model

The following cell checks the dataset path. If the data-location is already specified, then the notebook will use the dataset path mentioned in ai_reference_models.ini file. If the data-location is not specified in the ai_reference_models.ini, the data downloading option instructions are shown. You must manually download the dataset using these instructions before you can proceed to the next cell.

In [ ]:
import os
# Get the parameters from config
notebook_reference_root = os.environ.get('NotebookRoot')
%cd $notebook_reference_root
config = AIReferenceConfigFile(os.environ['yaml_file'])
config_data = config.read_config(model_1_name)
data_download_path=''

if config_data['data-download'] != '': #and config.data_location == '':
    print("\nFollow these instructions to get the data : ")
    if config_data['data-download'] != '':
        val = config_data['data-download']
    # use the "DATA_DOWNLOAD_PATH" environment variable value if it exists.
    env_data_download_path=os.environ.get('DATA_DOWNLOAD_PATH', '')
    if env_data_download_path != '':
        data_download_path= env_data_download_path

Set the path for dataset directory for the first model:

DATASET_DIR': the path where the dataset exists and is downloaded.

**ACTION : You need to input the path where the dataset for the first model exists or where you have downloaded it in your system**

In [ ]:
dataset_path = input('Input the path where the dataset exists for the first model:')

os.environ['DATASET_DIR'] = dataset_path

print("Data location path:", os.environ['DATASET_DIR'])

### Step 2.3: Prepare pre-trained model for the selected model

This step checks if the pre-trained model for the selected model exists in the pre-trained directory path. If the pre-trained directory does not exist, then it downloads the pre-trained model for the selected precision.

In [ ]:
config = AIReferenceConfigFile(os.environ['yaml_file'])

# Get the parameters from config
configvals=config.read_config(model_1_name)

# Iterate through the 'resnet50v1_5' list to find the matching 'precision' and get the 'wget' value
for item in configvals[model_1_name.split()[0]]:
    if item['precision'] == selected_precision:
        wget_value = item['wget']
        break  # You can break the loop once you find the match

# Get the pre-trained model file
pretrain_model_path = config.download_pretrained_model(wget=wget_value, current_path=current_path)
pretrain_model_path = config.uncompress_file(pretrain_model_path, current_path=current_path)
if pretrain_model_path:
    print('Downloaded the model in:', pretrain_model_path)
else:
    print('Failed to download the pretrained model.')

Set the environment variable for pre-trained model for the first model:

'PRETRAINED_MODEL': the path where is the pretrained_model exists and is downloaded.

NOTE: You can change the value of 'PRETRAINED_MODEL' by changing its assignment in the cell below.

In [ ]:
os.environ["PRETRAINED_MODEL"]=pretrain_model_path
print("Pretrain_model_path:", os.environ["PRETRAINED_MODEL"])

### Step 2.4: Set the log directory or output directory

'OUTPUT_DIR': the output directory path where model output logs are collected.

The default directory name for the output logs is \"logs\" in the current working directory. You can change this directory name by replacing the value assigned to log_directory in the cell below

In [ ]:
# Set the log directory/output directory to store the logs
log_directory=os.environ['NotebookRoot'] + os.sep + "logs"
print(log_directory)

#Set output-dir directory
if log_directory !='':
    configvals['output-dir'] = log_directory

os.environ['OUTPUT_DIR']=log_directory
print("Output directory path is:", os.environ['OUTPUT_DIR'])

### Step 2.5:  Run the first Model 

**if BATCH_SIZE is specified per model specification, set it for optimal batch size**

In [ ]:
if selected_section['set-batch-size']['cores']:
    import subprocess

    # Run the shell command to get the number of cores
    output = subprocess.check_output("lscpu | grep 'Core(s) per socket' | awk '{print $4}'", shell=True)

    # Decode the byte string output into a regular Python string
    cores = int(output.decode("utf-8").strip())

    cores = eval(selected_section['set-batch-size']['expr'])

**OPTIONAL: You can change the batch size from the model's default. For online_inference,  set batch_size value to be 1.** 

**Run the next cell if you want to change the batch-size, other skip running the next cell**

In [ ]:
batch_size= input('Set the value for batch size that you want to run: ')
os.environ["BATCH_SIZE"]=batch_size

Run the first model using the quickstart script configured in the ai_reference_models.ini file, and save output logs to the selected log directory.

In [ ]:
config = AIReferenceConfigFile(os.environ['yaml_file'])

# Get the parameters from config
config_data = config.read_config(model_1_name)
print(config_data)
print(config_data.get('ai-type'))

#print(config_dict.get('--mode:'))

# Split the model_name into individual parts
parts = model_1_name.split()

# Join the parts using hyphens as the separator
log_name = '-'.join(parts)
log_name = log_name + ".log"

ai_reference_root = os.environ.get('AIReferenceRoot')
%cd $ai_reference_root
run_workload = ("quickstart/" + config_data.get('ai-type') + "/" + config_data.get('framework') + "/"+ config_data.get('model-name') + "/"
                + config_data.get('mode') + "/" + config_data.get('device') +"/" + selected_script )
print(log_directory)
print(log_name)
print(run_workload)

import subprocess
print(selected_precision)
print(type(selected_precision))
cmd = "./{} {} {} | tee {}/{}".format(run_workload, selected_precision, "ipex-jit", log_directory, log_name)
print(cmd)
subprocess.call(cmd, shell=True)

%cd -

### Step 2.6 Get the throughput or accuracy of the first model

In [ ]:
# Throughput of first workload:
# Split the model_name into individual parts
parts_1 = model_1_name.split()

# Join the parts using hyphens as the separator
log_1_name = '-'.join(parts_1)
log_1_name = log_1_name + ".log"

import re, statistics
try:
    with open(f'{log_directory}/{log_1_name}', 'r') as file:
        print("DEBUGGING: FILE DIRECTORY:", f'{log_directory}/{log_1_name}')
        file_content = file.read()
        throughput_matches = re.findall(r'Throughput[^:]*:\s*(\d+\.\d+)\s+fps', file_content)
        accuracy_matches = re.findall(r'Accuracy[^:]*:\s*(\d+\.\d+)', file_content)
        if throughput_matches:
            throughput_matches = [float(x) for x in throughput_matches]
            throughput_value = float(statistics.mean(throughput_matches))
            print(f'Throughput: {throughput_value} fps')
        else:
            print('Throughput not found in the file.')
        if accuracy_matches:
            accuracy_matches = [float(x) for x in accuracy_matches]
            accuracy_value = float(statistics.mean(accuracy_matches))
            print(f'Accuracy: {accuracy_value}')
        else:
            print('Accuracy not found in the file')



except FileNotFoundError:
    print('File not found.')

## Step 3: Select and run the second model

Let's now run a second model and compare its performance with the first.

### Step 3.1 Select another model for comparision

After the list of models is displayed, select one by entering the model index number. (If the environment variable MODEL_2_INDEX is set, we'll use that instead of prompting for input.)

In [ ]:
notebook_root = os.environ['NotebookRoot']
%cd $notebook_root

In [ ]:
print("Supported Models: ")
print(models_table)

# use the "MODEL_2_INDEX" environment variable value if it exists.
import os
env_model_2_index=os.environ.get('MODEL_2_INDEX', '')
if env_model_2_index != '':
    model_2_index= int(env_model_2_index)
else:
    ## USER INPUT
    model_2_index= int(input('Input a index number of a model: '))

# List out the selected model name
if model_2_index >= len(sections):
    print("ERROR! Input a model_index value between 0 and ", len(sections))
else:
    model_2_name=sections[model_2_index]['name']

    # Prints out model name
    print("Second model is: ", model_2_name.split()[0] )

    selected_section = sections[model_2_index]
    model_name_2 = selected_section['model-name']

    # Check if the selected model has a precision section
    if model_name_2 in selected_section:
        print(f"Available Precisions for {model_name_2}:")

        # Create a table with headers
        precision_table = PrettyTable(["Index", "Precision"])
        precisions = selected_section[model_name_2]

        # Iterate through the available precisions
        for index, precision_info in enumerate(selected_section[model_name_2]):
            precision = precision_info['precision']

            # Add a row to the table
            precision_table.add_row([index, precision])

        print(precision_table)

        # Prompt the user to select a precision
        precision_index = int(input('Select an index number for the precision: '))
        selected_precision_2 = precisions[precision_index]['precision']
        print("Selected Precision for", model_2_name, selected_precision_2)

        # Print a table for available scripts for selected precision
        script_section = precisions[precision_index]
        script_name = script_section['script']

        print(f"Available Scripts for {selected_precision}:")

        # Create a table with headers
        script_table = PrettyTable(["Index", "Script"])

        # Iterate through the available scripts
        for index, script in enumerate(script_name):
            script_table.add_row([index, script])

        print(script_table)

        # Prompt the user to select a script
        script_index = int(input('Input a index number for the script: '))
        selected_row = script_table._rows[script_index]
        selected_script = selected_row[1]
        print("Selected script for", model_2_name, selected_script)

        # Prints out model name
        print("Second model with selected precision and script is: ", model_2_name, selected_precision_2, selected_script)
        os.environ['PRECISION']=selected_precision_2

### Additional Setup
The following cell checks for additional environment variables that needs to be specified per model specification. 

Then checks for additional files, libraries, and configurations that needs to be installed per model specification. Commands are installed in a chronological dependency order.

In [ ]:
import subprocess
ai_reference_root = os.environ.get('AIReferenceRoot')
%cd $ai_reference_root

for export in selected_section['exports']:
    env_name = export.split(' ')[0]
    env_value = export.split(' ')[1]
    if env_value == '-i':
        env_value = input(f'Please provide the path for the following environment variable {env_name}: ', )
    os.environ[env_name] = os.path.expandvars(env_value)

    print("Assigned environment variable ", env_name, " : ", os.environ[env_name])

for cmd in selected_section['additional-commands']:
    subprocess.call(cmd, shell=True)

### Step 3.2: Get the required dataset for the selected model

The following cell checks for the second dataset path. If the data-location is already specified, then the notebook will use the dataset path mentioned in ai_reference_models.ini file. If the data-location is not specified in the ai_reference_models.ini, the data downloading option instructions are shown. You must manually download the dataset using these instructions before you can proceed to the next cell.

In [ ]:
# Get the parameters from config
config = AIReferenceConfigFile(os.environ['yaml_file'])
config_data = config.read_config(model_2_name)
data_download_path=''

if config_data['data-download'] != '': #and config.data_location == '':
    print("\nFollow these instructions to get the data : ")
    if config_data['data-download'] != '':
        val = config_data['data-download']
    print(val)
    # use the "DATA_DOWNLOAD_PATH" environment variable value if it exists.
    env_data_download_path=os.environ.get('DATA_DOWNLOAD_PATH', '')
    if env_data_download_path != '':
        data_download_path= env_data_download_path

Set the path for dataset directory for the second model:

DATASET_DIR': the path where the dataset exists and is downloaded.

**ACTION : You need to input the path where the dataset for the second model exists or where you have downloaded it in your system**

In [ ]:
dataset_path = input('Input the path where the dataset exists for the second model:')

os.environ['DATASET_DIR'] = dataset_path
print("Data location path:", os.environ['DATASET_DIR'])

### Step 3.3: Prepare pre-trained model for the selected model

This step checks if the pre-trained model for the selected model exists in the pre-trained directory path. If the pre-trained directory does not exist, then it downloads the pre-trained model for the selected precision.

In [ ]:
config = AIReferenceConfigFile(os.environ['yaml_file'])

# Get the parameters from config
configvals=config.read_config(model_2_name)

# Iterate through the 'resnet50v1_5' list to find the matching 'precision' and get the 'wget' value
for item in configvals[model_2_name.split()[0]]:
    if item['precision'] == selected_precision_2:
        wget_value = item['wget']
        break  # You can break the loop once you find the match

# Get the pre-trained model file
pretrain_model_path = config.download_pretrained_model(wget=wget_value, current_path=current_path)
pretrain_model_path = config.uncompress_file(pretrain_model_path, current_path=current_path)
if pretrain_model_path:
    print('Downloaded the model in:', pretrain_model_path)
else:
    print('Failed to download the pretrained model.')

Set the environment variable for pre-trained model for the second model:

'PRETRAINED_MODEL': the path where is the pretrained_model exists and is downloaded.

NOTE: You can change the value of 'PRETRAINED_MODEL' by changing its assignment in the cell below.

In [ ]:
os.environ["PRETRAINED_MODEL"]=pretrain_model_path
print("Pretrain_model_path:", os.environ["PRETRAINED_MODEL"])

### Step 3.4: Run the second model

**if BATCH_SIZE is specified per model specification, set it for optimal batch size**

In [ ]:
if selected_section['set-batch-size']['cores']:
    import subprocess

    # Run the shell command to get the number of cores
    output = subprocess.check_output("lscpu | grep 'Core(s) per socket' | awk '{print $4}'", shell=True)

    # Decode the byte string output into a regular Python string
    cores = int(output.decode("utf-8").strip())

    cores = eval(selected_section['set-batch-size']['expr'])

**OPTIONAL: You can change the batch size from the model's default. For online_inference,  set batch_size value to be 1.** 

**Run the next cell if you want to change the batch-size, other skip running the next cell**

In [ ]:
batch_size= input('Set the value for batch size that you want to run: ')
os.environ["BATCH_SIZE"]=batch_size

Run the second model using the quickstart script configured in the ai_reference_models.ini file, and save output logs to the selected log directory.

In [ ]:
config = AIReferenceConfigFile(os.environ['yaml_file'])

# Get the parameters from config
config_data = config.read_config(model_2_name)

# Split the model_name into individual parts
parts = model_2_name.split()

# Join the parts using hyphens as the separator
log_name = '-'.join(parts)
log_name = log_name + ".log"

ai_reference_root = os.environ.get('AIReferenceRoot')

%cd $ai_reference_root
run_workload = ("quickstart/" + config_data.get('ai-type') + "/" + config_data.get('framework') + "/"+ config_data.get('model-name') + "/"
                + config_data.get('mode') + "/" + config_data.get('device') +"/" + selected_script )

!./$run_workload | tee $log_directory/{log_name}

%cd -

### Step 3.5 Get the throughput or accuracy of the second model

In [ ]:

# Throughput of second workload:
# Split the model_name into individual parts
parts_2 = model_2_name.split()

# Join the parts using hyphens as the separator
log_2_name = '-'.join(parts_2)
log_2_name = log_2_name + ".log"

import re, statistics
try:
    with open(f'{log_directory}/{log_2_name}', 'r') as file:
        print("DEBUGGING: FILE DIRECTORY:", f'{log_directory}/{log_2_name}')
        file_content = file.read()
        throughput_matches = re.findall(r'Throughput[^:]*:\s*(\d+\.\d+)\s+fps', file_content)
        accuracy_matches = re.findall(r'Accuracy[^:]*:\s*(\d+\.\d+)', file_content)
        if throughput_matches:
            throughput_matches = [float(x) for x in throughput_matches]
            throughput_value = float(statistics.mean(throughput_matches))
            print(f'Throughput: {throughput_value} fps')
        else:
            print('Throughput not found in the file.')
        if accuracy_matches:
            accuracy_matches = [float(x) for x in accuracy_matches]
            accuracy_value = float(statistics.mean(accuracy_matches))
            print(f'Accuracy: {accuracy_value}')
        else:
            print('Accuracy not found in the file')

## Step 4: Compare performance results and plot a comparison chart for the two models

Get the results (throughput/accuracy) of the two models for comparision

In [ ]:
print("Throughput of", model_1_name, ": ", Throughput_of_workload_1)
print("Throughput of", model_2_name, ": ", Throughput_of_workload_2)

Plot a chart for comparision 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
fig, ax = plt.subplots()

# Define your data
categories = [model_1_name.split(' ')[0] + selected_precision, model_2_name.split(' ')[0] + selected_precision_2]

# float array expected for charts.
values = [float(Throughput_of_workload_1), float(Throughput_of_workload_2)]


# Generate a list of colors for each bar
colors = ['blue', 'orange']


print(categories, values, colors)

ax.bar(x=categories, height=values, color=colors)
ax.set_ylabel("Throughput")
ax.set_title("Throughput by each model")
plt.show()